In [1]:

"""
Jupyter-ready Tkinter + SQLite To-Do & Tourism App
Features:
- Tkinter GUI with ttkbootstrap themes
- SQLite database (users, tasks, bookings)
- Login / Signup
- Tasks (add/delete/clear/export/import)
- Tourism panel: Cities (Mumbai,Pune,Goa,Delhi), Categories (Beaches,Hill Station,Waterfalls)
- Ticket booking: Bus, Train, Flight (saved to DB)
- Export bookings to JSON
- Jupyter usage notes: run `%gui tk` before launching or run the file with `!python main.py`

Save this file as main.py and run it from Jupyter:
1) Install dependency: pip install ttkbootstrap
2) In a notebook cell run:
   %gui tk
   !python main.py
or import and launch directly:
   %gui tk
   from main import TodoApp
   app = TodoApp()
   app.mainloop()
"""
import json
import os
import sqlite3
import time
import ttkbootstrap as tb
from ttkbootstrap.constants import *
from ttkbootstrap import StringVar, IntVar
import tkinter as tk
from tkinter import filedialog, messagebox, ttk

DB_FILE = "todo_app.db"
EXPORT_DIR = "exports"

# ------------------ Config choices (as user requested) ------------------
CITIES = ["Mumbai", "Pune", "Goa", "Delhi"]
CATEGORIES = ["Beaches", "Hill Station", "Waterfalls"]
TICKET_TYPES = ["Bus", "Train", "Flight"]

# Simple mapping of places per (city, category)
PLACES_MAP = {
    ("Mumbai", "Beaches"): ["Juhu Beach", "Girgaun Chowpatty", "Aksa Beach"],
    ("Goa", "Beaches"): ["Baga Beach", "Calangute", "Palolem"],
    ("Pune", "Hill Station"): ["Lonavala", "Khandala"],
    ("Delhi", "Historical Monuments"): ["Red Fort", "Qutub Minar", "India Gate"],
    ("Mumbai", "Historical Monuments"): ["Gateway of India", "Elephanta Caves"],
    ("Goa", "Hill Station"): ["Dudhsagar (nearby)"],
    ("Pune", "Waterfalls"): ["Kune Falls", "Lonavala Falls"],
    ("Goa", "Waterfalls"): ["Dudhsagar Falls"],
    ("Mumbai", "Waterfalls"): ["Tungareshwar Falls (seasonal)"],
    ("Delhi", "Beaches"): [],
    ("Pune", "Beaches"): [],
    ("Delhi", "Hill Station"): [],
}

# --------------------- Database helpers ---------------------
class Database:
    def __init__(self, db_file=DB_FILE):
        self.conn = sqlite3.connect(db_file)
        self.create_tables()

    def create_tables(self):
        c = self.conn.cursor()
        c.execute(
            """
            CREATE TABLE IF NOT EXISTS users (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                username TEXT UNIQUE NOT NULL,
                password TEXT NOT NULL
            )
            """
        )
        c.execute(
            """
            CREATE TABLE IF NOT EXISTS tasks (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                user_id INTEGER NOT NULL,
                title TEXT NOT NULL,
                done INTEGER DEFAULT 0,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                FOREIGN KEY(user_id) REFERENCES users(id)
            )
            """
        )
        c.execute(
            """
            CREATE TABLE IF NOT EXISTS bookings (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                user_id INTEGER NOT NULL,
                city TEXT NOT NULL,
                category TEXT NOT NULL,
                place TEXT NOT NULL,
                ticket_type TEXT NOT NULL,
                travel_date TEXT,
                passengers INTEGER DEFAULT 1,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                FOREIGN KEY(user_id) REFERENCES users(id)
            )
            """
        )
        self.conn.commit()

    # user methods
    def add_user(self, username, password):
        try:
            c = self.conn.cursor()
            c.execute("INSERT INTO users (username, password) VALUES (?, ?)", (username, password))
            self.conn.commit()
            return True
        except sqlite3.IntegrityError:
            return False

    def check_user(self, username, password):
        c = self.conn.cursor()
        c.execute("SELECT id FROM users WHERE username = ? AND password = ?", (username, password))
        row = c.fetchone()
        return row[0] if row else None

    # task methods
    def get_tasks(self, user_id):
        c = self.conn.cursor()
        c.execute("SELECT id, title, done FROM tasks WHERE user_id = ? ORDER BY id", (user_id,))
        return c.fetchall()

    def add_task(self, user_id, title):
        c = self.conn.cursor()
        c.execute("INSERT INTO tasks (user_id, title) VALUES (?,?)", (user_id, title))
        self.conn.commit()
        return c.lastrowid

    def delete_task(self, task_id):
        c = self.conn.cursor()
        c.execute("DELETE FROM tasks WHERE id = ?", (task_id,))
        self.conn.commit()

    def set_done(self, task_id, done):
        c = self.conn.cursor()
        c.execute("UPDATE tasks SET done = ? WHERE id = ?", (1 if done else 0, task_id))
        self.conn.commit()

    # booking methods
    def add_booking(self, user_id, city, category, place, ticket_type, travel_date, passengers):
        c = self.conn.cursor()
        c.execute(
            "INSERT INTO bookings (user_id, city, category, place, ticket_type, travel_date, passengers) VALUES (?,?,?,?,?,?,?)",
            (user_id, city, category, place, ticket_type, travel_date, passengers),
        )
        self.conn.commit()
        return c.lastrowid

    def get_bookings(self, user_id):
        c = self.conn.cursor()
        c.execute(
            "SELECT id, city, category, place, ticket_type, travel_date, passengers FROM bookings WHERE user_id = ? ORDER BY id",
            (user_id,),
        )
        return c.fetchall()

    def delete_booking(self, booking_id):
        c = self.conn.cursor()
        c.execute("DELETE FROM bookings WHERE id = ?", (booking_id,))
        self.conn.commit()

    def close(self):
        self.conn.close()


# --------------------- Login / Signup Window ---------------------
class AuthWindow(tk.Toplevel):
    def __init__(self, master, db, on_success):
        super().__init__(master)
        self.db = db
        self.on_success = on_success
        self.title("Login / Signup")
        self.geometry("360x300")
        self.resizable(False, False)
        self.protocol("WM_DELETE_WINDOW", self.on_close)

        self.style = tb.Style()

        self.frame = tb.Frame(self, padding=12)
        self.frame.pack(expand=True, fill="both")

        tb.Label(self.frame, text="Welcome", font=(None, 18, "bold")).pack(pady=(0, 8))

        self.tab = ttk.Notebook(self.frame)
        self.login_tab = tb.Frame(self.tab)
        self.signup_tab = tb.Frame(self.tab)
        self.tab.add(self.login_tab, text="Login")
        self.tab.add(self.signup_tab, text="Signup")
        self.tab.pack(expand=True, fill="both")

        # Login
        tb.Label(self.login_tab, text="Username").pack(anchor="w")
        self.login_user = tb.Entry(self.login_tab)
        self.login_user.pack(fill="x", pady=4)
        tb.Label(self.login_tab, text="Password").pack(anchor="w")
        self.login_pass = tb.Entry(self.login_tab, show="*")
        self.login_pass.pack(fill="x", pady=4)
        tb.Button(self.login_tab, text="Login", bootstyle=PRIMARY, command=self.do_login).pack(pady=8)

        # Signup
        tb.Label(self.signup_tab, text="Choose username").pack(anchor="w")
        self.signup_user = tb.Entry(self.signup_tab)
        self.signup_user.pack(fill="x", pady=4)
        tb.Label(self.signup_tab, text="Choose password").pack(anchor="w")
        self.signup_pass = tb.Entry(self.signup_tab, show="*")
        self.signup_pass.pack(fill="x", pady=4)
        tb.Button(self.signup_tab, text="Signup", bootstyle=SUCCESS, command=self.do_signup).pack(pady=8)

    def do_login(self):
        u = self.login_user.get().strip()
        p = self.login_pass.get().strip()
        if not u or not p:
            messagebox.showwarning("Warning", "Enter username and password")
            return
        user_id = self.db.check_user(u, p)
        if user_id:
            self.on_success(user_id, u)
            self.destroy()
        else:
            messagebox.showerror("Error", "Invalid credentials")

    def do_signup(self):
        u = self.signup_user.get().strip()
        p = self.signup_pass.get().strip()
        if not u or not p:
            messagebox.showwarning("Warning", "Enter username and password")
            return
        ok = self.db.add_user(u, p)
        if ok:
            messagebox.showinfo("Success", "Account created! You can login now.")
            self.tab.select(self.login_tab)
        else:
            messagebox.showerror("Error", "Username already exists")

    def on_close(self):
        # if user closes auth window, exit app
        self.master.destroy()


# --------------------- Tourism Panel ---------------------
class TourismPanel(tb.Frame):
    def __init__(self, master, db, current_user_id, *args, **kwargs):
        super().__init__(master, *args, **kwargs)
        self.db = db
        self.current_user_id = current_user_id
        self.pack(fill="both", expand=True, padx=6, pady=6)

        top = tb.Frame(self)
        top.pack(fill="x", pady=(0,8))

        tb.Label(top, text="Tourism / Booking", font=(None, 14, "bold")).pack(side="left")

        # Controls frame
        controls = tb.Frame(self)
        controls.pack(fill="x", pady=6)

        tb.Label(controls, text="City:").grid(row=0, column=0, sticky="w")
        self.city_var = StringVar(value=CITIES[0])
        self.city_cb = ttk.Combobox(controls, values=CITIES, textvariable=self.city_var, state="readonly", width=20)
        self.city_cb.grid(row=0, column=1, padx=6, pady=4)

        tb.Label(controls, text="Category:").grid(row=1, column=0, sticky="w")
        self.category_var = StringVar(value=CATEGORIES[0])
        self.category_cb = ttk.Combobox(controls, values=CATEGORIES, textvariable=self.category_var, state="readonly", width=20)
        self.category_cb.grid(row=1, column=1, padx=6, pady=4)

        tb.Label(controls, text="Places:").grid(row=2, column=0, sticky="nw")
        self.places_list = tk.Listbox(controls, height=5, width=30)
        self.places_list.grid(row=2, column=1, padx=6, pady=4, sticky="w")

        tb.Label(controls, text="Ticket Type:").grid(row=3, column=0, sticky="w")
        self.ticket_var = StringVar(value=TICKET_TYPES[0])
        for i, t in enumerate(TICKET_TYPES):
            rb = tb.Radiobutton(controls, text=t, variable=self.ticket_var, value=t)
            rb.grid(row=3, column=1+i, padx=(0,6), sticky="w")

        tb.Label(controls, text="Travel Date (YYYY-MM-DD):").grid(row=4, column=0, sticky="w")
        self.date_entry = tb.Entry(controls, width=22)
        self.date_entry.grid(row=4, column=1, pady=4, sticky="w")

        tb.Label(controls, text="Passengers:").grid(row=5, column=0, sticky="w")
        self.pass_spin = tk.Spinbox(controls, from_=1, to=10, width=5)
        self.pass_spin.grid(row=5, column=1, sticky="w")

        tb.Button(controls, text="Show Places", bootstyle=PRIMARY, command=self.show_places).grid(row=6, column=1, pady=6, sticky="w")
        tb.Button(controls, text="Book Ticket", bootstyle=SUCCESS, command=self.book_ticket).grid(row=6, column=1, pady=6, sticky="e")

        # Bookings table
        self.tree = ttk.Treeview(self, columns=("id","city","category","place","ticket","date","pax"), show="headings", height=8)
        for col, heading in [("id","ID"),("city","City"),("category","Category"),("place","Place"),("ticket","Ticket"),("date","Date"),("pax","Passengers")]:
            self.tree.heading(col, text=heading)
            self.tree.column(col, width=100, anchor="center")
        self.tree.pack(fill="both", expand=True, pady=8)

        btns = tb.Frame(self)
        btns.pack(fill="x")
        tb.Button(btns, text="Refresh Bookings", bootstyle=INFO, command=self.load_bookings).pack(side="left", padx=6)
        tb.Button(btns, text="Export Bookings", bootstyle=SECONDARY, command=self.export_bookings).pack(side="left")
        tb.Button(btns, text="Delete Booking", bootstyle=DANGER, command=self.delete_selected_booking).pack(side="right", padx=6)

        # initial population
        self.show_places()
        self.load_bookings()

    def show_places(self):
        city = self.city_var.get()
        cat = self.category_var.get()
        key = (city, cat)
        places = PLACES_MAP.get(key, [])
        self.places_list.delete(0, tk.END)
        if not places:
            # fallback: search in PLACES_MAP keys with matching category
            fallback = []
            for (c,k), v in PLACES_MAP.items():
                if k == cat and v:
                    fallback.extend(v)
            places = fallback[:5]
        for p in places:
            self.places_list.insert(tk.END, p)

    def book_ticket(self):
        sel = self.places_list.curselection()
        if not sel:
            messagebox.showwarning("Warning", "Select a place from the list (click 'Show Places' first)")
            return
        place = self.places_list.get(sel[0])
        city = self.city_var.get()
        cat = self.category_var.get()
        ticket = self.ticket_var.get()
        date = self.date_entry.get().strip()
        try:
            pax = int(self.pass_spin.get())
        except:
            pax = 1
        if not date:
            if not messagebox.askyesno("No date", "No travel date entered. Continue without date?"):
                return
        # save booking
        if not self.current_user_id:
            messagebox.showerror("Error", "You must be logged in to book")
            return
        bid = self.db.add_booking(self.current_user_id, city, cat, place, ticket, date, pax)
        messagebox.showinfo("Booked", f"Booking saved (ID {bid})")
        self.load_bookings()

    def load_bookings(self):
        for row in self.tree.get_children():
            self.tree.delete(row)
        if not self.current_user_id:
            return
        rows = self.db.get_bookings(self.current_user_id)
        for r in rows:
            self.tree.insert("", tk.END, values=r)

    def export_bookings(self):
        if not self.current_user_id:
            messagebox.showerror("Error", "Login to export bookings")
            return
        rows = self.db.get_bookings(self.current_user_id)
        bookings = []
        for r in rows:
            bookings.append({
                "id": r[0],
                "city": r[1],
                "category": r[2],
                "place": r[3],
                "ticket_type": r[4],
                "travel_date": r[5],
                "passengers": r[6]
            })
        if not os.path.exists(EXPORT_DIR):
            os.makedirs(EXPORT_DIR)
        filename = os.path.join(EXPORT_DIR, f"bookings_{self.current_user_id}_{int(time.time())}.json")
        with open(filename, "w", encoding="utf-8") as f:
            json.dump(bookings, f, ensure_ascii=False, indent=2)
        messagebox.showinfo("Exported", f"Bookings exported to {filename}")

    def delete_selected_booking(self):
        sel = self.tree.selection()
        if not sel:
            messagebox.showwarning("Warning", "Select a booking to delete")
            return
        item = self.tree.item(sel[0])
        bid = item["values"][0]
        if messagebox.askyesno("Confirm", f"Delete booking ID {bid}?"):
            self.db.delete_booking(bid)
            self.load_bookings()


# --------------------- Main Application ---------------------
class TodoApp(tb.Window):
    def __init__(self):
        super().__init__(themename="flatly")
        self.title("To-Do App — Tkinter + SQLite + Tourism")
        self.geometry("980x640")
        self.minsize(900, 600)

        # Database
        self.db = Database()
        self.current_user_id = None
        self.current_username = None

        # Top frame controls
        top = tb.Frame(self, padding=10)
        top.pack(side="top", fill="x")

        self.title_lbl = tb.Label(top, text="Tk To-Do App", font=(None, 18, "bold"))
        self.title_lbl.pack(side="left")

        # Theme toggle
        self.style = self.style  # tb.Style() available as self.style
        self.theme_btn = tb.Button(top, text="Toggle Dark/Light", bootstyle=INFO, command=self.toggle_theme)
        self.theme_btn.pack(side="right", padx=6)

        # User label + logout
        self.user_lbl = tb.Label(top, text="Not logged in")
        self.user_lbl.pack(side="right", padx=10)
        self.logout_btn = tb.Button(top, text="Logout", bootstyle=DANGER, command=self.logout)
        self.logout_btn.pack(side="right")
        self.logout_btn.pack_forget()  # hidden until login

        # Tourism button
        self.tourism_btn = tb.Button(top, text="Open Tourism Panel", bootstyle=PRIMARY, command=self.open_tourism_panel)
        self.tourism_btn.pack(side="right", padx=6)

        # Main layout
        main = tb.Frame(self, padding=(10, 10))
        main.pack(expand=True, fill="both")

        left = tb.Frame(main)
        left.pack(side="left", fill="y", padx=(0, 10))

        right = tb.Frame(main)
        right.pack(side="left", expand=True, fill="both")

        # Left: Controls (Tasks)
        tb.Label(left, text="Add Task:").pack(anchor="w")
        self.task_var = StringVar()
        self.task_entry = tb.Entry(left, textvariable=self.task_var, width=30)
        self.task_entry.pack(pady=6)
        tb.Button(left, text="Add", bootstyle=SUCCESS, command=self.add_task_animated).pack(fill="x")
        tb.Button(left, text="Delete Selected", bootstyle=WARNING, command=self.delete_selected).pack(fill="x", pady=6)
        tb.Button(left, text="Clear All", bootstyle=SECONDARY, command=self.clear_all_confirm).pack(fill="x")
        tb.Button(left, text="Save to File", bootstyle=PRIMARY, command=self.export_tasks).pack(fill="x", pady=6)
        tb.Button(left, text="Load from File", bootstyle=INFO, command=self.import_tasks).pack(fill="x")

        # right: task list (scrollable)
        self.tasks_frame = tb.Frame(right)
        self.tasks_frame.pack(expand=True, fill="both")

        self.tasks_canvas = tk.Canvas(self.tasks_frame, borderwidth=0, highlightthickness=0)
        self.tasks_scroll = tb.Scrollbar(self.tasks_frame, orient="vertical", command=self.tasks_canvas.yview)
        self.tasks_inner = tb.Frame(self.tasks_canvas)

        self.tasks_inner.bind(
            "<Configure>",
            lambda e: self.tasks_canvas.configure(scrollregion=self.tasks_canvas.bbox("all")),
        )

        self.tasks_canvas.create_window((0, 0), window=self.tasks_inner, anchor="nw")
        self.tasks_canvas.configure(yscrollcommand=self.tasks_scroll.set)

        self.tasks_canvas.pack(side="left", fill="both", expand=True)
        self.tasks_scroll.pack(side="right", fill="y")

        # footer status
        self.status_lbl = tb.Label(self, text="Please login to get started", bootstyle="secondary")
        self.status_lbl.pack(side="bottom", fill="x")

        # Tourism panel attribute
        self.tourism_panel = None

        # If not logged in — show auth window
        self.after(200, self.show_auth)

    # ----------------- Authentication -----------------
    def show_auth(self):
        def on_success(user_id, username):
            self.current_user_id = user_id
            self.current_username = username
            self.user_lbl.config(text=f"User: {username}")
            self.logout_btn.pack(side="right")
            self.status_lbl.config(text=f"Logged in as {username}")
            self.load_tasks()
            if self.tourism_panel:
                self.tourism_panel.current_user_id = self.current_user_id
                self.tourism_panel.load_bookings()

        AuthWindow(self, self.db, on_success)

    def logout(self):
        self.current_user_id = None
        self.current_username = None
        self.user_lbl.config(text="Not logged in")
        self.logout_btn.pack_forget()
        self.clear_task_widgets()
        self.status_lbl.config(text="Logged out.")
        # Show auth again
        self.after(200, self.show_auth)

    # ----------------- Theme -----------------
    def toggle_theme(self):
        current = self.style.theme_use()
        dark = "darkly"
        light = "flatly"
        new = dark if current != dark else light
        self.style.theme_use(new)
        self.status_lbl.config(text=f"Theme set to {new}")

    # ----------------- Tasks management -----------------
    def clear_task_widgets(self):
        for child in self.tasks_inner.winfo_children():
            child.destroy()

    def load_tasks(self):
        self.clear_task_widgets()
        if not self.current_user_id:
            return
        rows = self.db.get_tasks(self.current_user_id)
        for task in rows:
            self._create_task_row(task_id=task[0], title=task[1], done=bool(task[2]))
        self._attach_selection()
        self.status_lbl.config(text=f"{len(rows)} tasks loaded for {self.current_username}")

    def _create_task_row(self, task_id, title, done=False):
        frame = tb.Frame(self.tasks_inner, bootstyle="info-subtle", padding=6)
        frame.pack(fill="x", pady=4, padx=6)

        var = IntVar(value=1 if done else 0)
        chk = tb.Checkbutton(frame, variable=var, bootstyle="success", command=lambda tid=task_id, v=var: self.toggle_done(tid, v))
        chk.pack(side="left")

        lbl = tb.Label(frame, text=title, font=(None, 12))
        lbl.pack(side="left", padx=8)

        # store id on widget for deletion/selection
        frame.task_id = task_id

        # click to select
        frame.bind("<Button-1>", lambda e, w=frame: self._on_task_click(w))
        lbl.bind("<Button-1>", lambda e, w=frame: self._on_task_click(w))

        return frame

    def add_task_animated(self):
        title = self.task_var.get().strip()
        if not title:
            messagebox.showwarning("Warning", "Enter a task")
            return
        if not self.current_user_id:
            messagebox.showerror("Error", "You must be logged in")
            return
        task_id = self.db.add_task(self.current_user_id, title)
        frame = self._create_task_row(task_id, title, done=False)
        self.task_var.set("")
        self.status_lbl.config(text="Task added")
        self._flash_widget(frame)

    def _flash_widget(self, widget, flashes=6, delay=120):
        def step(i):
            if i <= 0:
                widget.configure(relief="flat")
                return
            widget.configure(relief="raised" if i % 2 == 0 else "flat")
            self.after(delay, lambda: step(i - 1))
        step(flashes)

    def delete_selected(self):
        selected = None
        for child in self.tasks_inner.winfo_children():
            if getattr(child, "selected", False):
                selected = child
                break
        if not selected:
            messagebox.showwarning("Warning", "Select a task (click on it) to delete")
            return
        task_id = selected.task_id
        if messagebox.askyesno("Confirm", "Delete selected task?"):
            self.db.delete_task(task_id)
            selected.destroy()
            self.status_lbl.config(text="Task deleted")

    def clear_all_confirm(self):
        if not self.current_user_id:
            messagebox.showerror("Error", "You must be logged in")
            return
        if messagebox.askyesno("Confirm", "Delete ALL tasks for this user? This cannot be undone."):
            rows = self.db.get_tasks(self.current_user_id)
            for r in rows:
                self.db.delete_task(r[0])
            self.clear_task_widgets()
            self.status_lbl.config(text="All tasks cleared")

    def toggle_done(self, task_id, var):
        val = bool(var.get())
        self.db.set_done(task_id, val)
        self.status_lbl.config(text=("Task marked done" if val else "Task marked not done"))

    # ----------------- Export / Import -----------------
    def export_tasks(self):
        if not self.current_user_id:
            messagebox.showerror("Error", "Login first to export tasks")
            return
        rows = self.db.get_tasks(self.current_user_id)
        tasks = [{"id": r[0], "title": r[1], "done": bool(r[2])} for r in rows]
        if not os.path.exists(EXPORT_DIR):
            os.makedirs(EXPORT_DIR)
        filename = os.path.join(EXPORT_DIR, f"tasks_{self.current_username}_{int(time.time())}.json")
        with open(filename, "w", encoding="utf-8") as f:
            json.dump(tasks, f, ensure_ascii=False, indent=2)
        messagebox.showinfo("Exported", f"Tasks exported to {filename}")
        self.status_lbl.config(text=f"Exported {len(tasks)} tasks")

    def import_tasks(self):
        if not self.current_user_id:
            messagebox.showerror("Error", "Login first to import tasks")
            return
        file = filedialog.askopenfilename(title="Select JSON file", filetypes=[("JSON Files", "*.json")])
        if not file:
            return
        try:
            with open(file, "r", encoding="utf-8") as f:
                data = json.load(f)
            count = 0
            for item in data:
                title = item.get("title") or item.get("task")
                if title:
                    self.db.add_task(self.current_user_id, title)
                    count += 1
            self.load_tasks()
            messagebox.showinfo("Imported", f"Imported {count} tasks")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to import: {e}")

    # ----------------- Selection handling -----------------
    def _attach_selection(self):
        for child in self.tasks_inner.winfo_children():
            child.bind("<Button-1>", lambda e, w=child: self._on_task_click(w))

    def _on_task_click(self, widget):
        for child in self.tasks_inner.winfo_children():
            child.selected = False
            child.configure(style=None)
        widget.selected = True
        widget.configure(style="primary.TFrame")

    # ----------------- Tourism Panel control -----------------
    def open_tourism_panel(self):
        if self.tourism_panel:
            # bring to front (destroy and recreate to ensure updated user id)
            self.tourism_panel.destroy()
            self.tourism_panel = None
        self.tourism_panel = TourismPanel(self, self.db, self.current_user_id)

    def on_close(self):
        # close db connection gracefully
        self.db.close()
        self.destroy()


# --------------------- Run ---------------------
if __name__ == '__main__':
    # Ensure exports dir
    if not os.path.exists(EXPORT_DIR):
        os.makedirs(EXPORT_DIR)

    app = TodoApp()
    app.protocol("WM_DELETE_WINDOW", app.on_close)
    app.mainloop()


ModuleNotFoundError: No module named 'ttkbootstrap'

In [2]:
!pip install ttkbootstrap


In [3]:
%gui tk
!python main.py


python: can't open file 'C:\\Users\\mayur\\OneDrive\\Desktop\\gen ai 6 month\\main.py': [Errno 2] No such file or directory
